In [3]:
#Define working directory
import os
os.chdir('/Users/LMWee/Sensor_design/Long_Design_Shorten/')
os.getcwd()

'/Users/LMWee/Sensor_design/Long_Design_Shorten'

In [5]:
# Jupyter-ready: mRNA -> designs pipeline (searches CCA, produces FASTA + CSV + skipped log)
from Bio import SeqIO
from Bio.Seq import Seq
from Bio.SeqRecord import SeqRecord
import pandas as pd
import re

# -----------------------------
# User parameters / filenames
# -----------------------------
input_fasta = "Trat1.fasta"      # your input file
output_fasta = "Trat1_designs.fasta"        # final accepted designs
output_csv = "Trat1_designs.csv"            # CSV with columns: label, full_seq, aa_seq, region_class
skipped_log = "Trat1_skipped_designs.log"   # human-readable log for skipped designs

# Default adapters (user may override by setting user_prefix/user_suffix)
default_prefix = "CCTGGGCCGATTAAGGGCCTGCAGGGTGGA"
default_suffix = "TCTACTCGAGGGGCTAGCCAGGGCAGCGGC"
user_prefix = None
user_suffix = None

# MS2 loop inserts (uppercase DNA)
MS2_inserts = {
    21: "AGACATGAGGATCACCCATGT",   # variant 1
    47: "GGGTGGAGGAACACCCCACCC",   # variant 2
    103: "AGAAGCACCATCAGGGCTTCT",  # variant 3
    129: "GCGTGGAGCATCAGCCCACGC"   # variant 4
}

# BspQI / SapI recognition sequences (DNA-level)
BspQI_sites = ["GCTCTTC", "GAAGAGC"]
SapI_sites = ["GCTCTTC", "GAAGAGC"]
FORBIDDEN_SITES = BspQI_sites + SapI_sites

# -----------------------------
# Helper functions
# -----------------------------
def check_adapter_validity(adapter_seq: str, adapter_name="Adapter"):
    """Validate adapter: length multiple of 3, no in-frame stop (frame 0), no forbidden sites."""
    adapter_seq = adapter_seq.upper()
    if len(adapter_seq) % 3 != 0:
        raise ValueError(f"{adapter_name} length ({len(adapter_seq)}) is not a multiple of 3")
    # check in-frame stop codons in frame 0
    for i in range(0, len(adapter_seq)-2, 3):
        codon = adapter_seq[i:i+3]
        if codon in ("TAG", "TAA", "TGA"):
            raise ValueError(f"{adapter_name} contains stop codon {codon} at position {i} (frame 0)")
    # check forbidden sites anywhere in adapter
    for site in FORBIDDEN_SITES:
        if site in adapter_seq:
            raise ValueError(f"{adapter_name} contains forbidden site {site}")
    return True

def dna_standardize(seq: str) -> str:
    return str(seq).upper().replace("U", "T")

def reverse_complement(seq: str) -> str:
    return str(Seq(seq).reverse_complement())

def replace_central(seq: str, new_codon="TAG", central_pos=76) -> str:
    return seq[:central_pos] + new_codon + seq[central_pos+3:]

def insert_MS2_loops(seq: str, MS2_dict: dict) -> str:
    """Insert all MS2 loops in one pass using offsets to account for growth."""
    s = seq
    offset = 0
    for pos in sorted(MS2_dict.keys()):
        insert = MS2_dict[pos].upper()
        s = s[:pos+offset] + insert + s[pos+offset+5:]  # replace 5 nt at pos with 21-nt insert
        offset += len(insert) - 5
    return s

def replace_in_frame_stops(seq: str, central_TAG_pos: int) -> str:
    """Replace other in-frame stops (frame 0) except the central TAG at central_TAG_pos."""
    seq_list = list(seq)
    for i in range(0, len(seq)-2, 3):
        if i == central_TAG_pos:
            continue
        codon = "".join(seq_list[i:i+3])
        if codon == "TAG":
            seq_list[i:i+3] = list("CAG")
        elif codon == "TAA":
            seq_list[i:i+3] = list("TCA")
        elif codon == "TGA":
            seq_list[i:i+3] = list("GGA")
    return "".join(seq_list)

def replace_in_frame_ATG(seq: str) -> str:
    """Replace in-frame ATG -> ATT (frame 0)."""
    seq_list = list(seq)
    for i in range(0, len(seq)-2, 3):
        codon = "".join(seq_list[i:i+3])
        if codon == "ATG":
            seq_list[i:i+3] = list("ATT")
    return "".join(seq_list)

def contains_forbidden_sites(seq: str) -> bool:
    for site in FORBIDDEN_SITES:
        if site in seq:
            return True
    return False

def translate_seq(full_seq: str, central_tag_pos: int = None) -> str:
    """Translate full_seq in-frame (frame 0). Mark central TAG (if present) as '*' at given index."""
    aa = []
    for i in range(0, len(full_seq)-2, 3):
        codon = full_seq[i:i+3]
        if central_tag_pos is not None and i == central_tag_pos:
            aa.append("*")
        else:
            aa.append(str(Seq(codon).translate()))
    return "".join(aa)

def classify_region(start_pos: int, seq_len: int, X: int, Y: int, Z: int) -> str:
    """Classify where the extracted window (start_pos ... start_pos+seq_len-1) maps to [X,Y,Z]."""
    end_pos = start_pos + seq_len - 1
    U5_end = X - 1
    ORF_start = X
    ORF_end = X + Y - 1
    U3_start = X + Y
    U3_end = X + Y + Z - 1

    # booleans for overlap - use inclusive coords
    in_5U = start_pos >= 0 and end_pos <= U5_end
    in_5U_ORF = start_pos <= U5_end and end_pos >= ORF_start and end_pos <= ORF_end
    in_ORF = start_pos >= ORF_start and end_pos <= ORF_end
    in_ORF_3U = start_pos >= ORF_start and start_pos <= ORF_end and end_pos >= U3_start
    in_3U = start_pos >= U3_start and end_pos <= U3_end
    in_all = start_pos <= U5_end and end_pos >= U3_start

    if in_5U:
        return "5U"
    elif in_5U_ORF:
        return "5U-ORF"
    elif in_ORF:
        return "ORF"
    elif in_ORF_3U:
        return "ORF-3U"
    elif in_3U:
        return "3U"
    elif in_all:
        return "5U-ORF-3U"
    else:
        return "UNCLASSIFIED"

# -----------------------------
# Prepare adapters
# -----------------------------
prefix = (user_prefix.upper() if user_prefix else default_prefix.upper())
suffix = (user_suffix.upper() if user_suffix else default_suffix.upper())
# Validate adapters (will raise if invalid)
check_adapter_validity(prefix, "Prefix")
check_adapter_validity(suffix, "Suffix")

# -----------------------------
# Main processing loop
# -----------------------------
design_records = []
csv_records = []
skipped_records = []

for record in SeqIO.parse(input_fasta, "fasta"):
    rna_name = record.id                     # e.g., Trat1 (used for label)
    description = record.description         # full header line, contains [X,Y,Z]
    seq_orig = dna_standardize(str(record.seq))

    # Parse X,Y,Z from description - allow optional spaces
    match = re.search(r"\[(\d+)\s*,\s*(\d+)\s*,\s*(\d+)\s*\]", description)
    if not match:
        skipped_records.append(f"{rna_name}: missing [X,Y,Z] annotation")
        continue
    X, Y, Z = map(int, match.groups())

    # Find all CCA sites
    for m in re.finditer("CCA", seq_orig):
        start_c = m.start()  # 0-indexed position of the first 'C' in CCA in the original mRNA

        # Extract ±76 nt (76 left, CCA, 76 right) => total 155 nt
        left = start_c - 76
        right = start_c + 3 + 76
        if left < 0 or right > len(seq_orig):
            skipped_records.append(f"{rna_name}_{start_c}: incomplete flanking sequences")
            continue
        sub_seq = seq_orig[left:right]  # length should be 155
        if len(sub_seq) != 155:
            skipped_records.append(f"{rna_name}_{start_c}: extracted length != 155 ({len(sub_seq)})")
            continue

        # Region classification (based on where this 155-nt window starts in original mRNA)
        region_class = classify_region(left, len(sub_seq), X, Y, Z)

        # Reverse complement => central triplet should be TGG (revcomp of CCA)
        sub_seq = reverse_complement(sub_seq)
        if sub_seq[76:79] != "TGG":
            # Unexpected; skip and log (ensures central TGG as requested)
            skipped_records.append(f"{rna_name}_{start_c}: central codon not TGG after reverse complement (found '{sub_seq[76:79]}')")
            continue

        # Change central TGG -> TAG at positions 76-78 (0-indexed)
        sub_seq = replace_central(sub_seq, new_codon="TAG", central_pos=76)

        # Insert MS2 loops at positions 21,47,103,129 (single-step with offset)
        sub_seq = insert_MS2_loops(sub_seq, MS2_inserts)

        # After insertions, central TAG should be at 108 (0-indexed)
        central_TAG_pos = 108

        # Replace other in-frame stops (frame 0) except central TAG
        sub_seq = replace_in_frame_stops(sub_seq, central_TAG_pos)

        # Replace in-frame ATG -> ATT
        sub_seq = replace_in_frame_ATG(sub_seq)

        # Append adapters
        full_seq = prefix + sub_seq + suffix

        # Final forbidden-site check (BspQI/SapI)
        if contains_forbidden_sites(full_seq):
            skipped_records.append(f"{rna_name}_{start_c}: BspQI/SapI site detected after modifications")
            continue

        # Translate sequence and mark central TAG as '*' (central position in full_seq)
        aa_seq = translate_seq(full_seq, central_tag_pos = len(prefix) + central_TAG_pos)

        # Label format requested: f"{rna_name}_{idx}" where idx is 0-indexed first C of CCA
        label = f"{rna_name}_{start_c}"

        # Save sequences and csv metadata
        design_records.append(SeqRecord(Seq(full_seq), id=label, description=""))
        csv_records.append({
            "label": label,
            "full_seq": full_seq,
            "aa_seq": aa_seq,
            "region_class": region_class
        })

# -----------------------------
# Write outputs
# -----------------------------
# FASTA (accepted designs)
SeqIO.write(design_records, output_fasta, "fasta")

# CSV with columns: label, full_seq, aa_seq, region_class
df = pd.DataFrame(csv_records, columns=["label","full_seq","aa_seq","region_class"])
df.to_csv(output_csv, index=False)

# Skipped log
with open(skipped_log, "w") as fh:
    for line in skipped_records:
        fh.write(line + "\n")

print(f"Processed designs: {len(design_records)}")
print(f"Skipped entries: {len(skipped_records)} (see {skipped_log})")
print(f"Wrote FASTA: {output_fasta}")
print(f"Wrote CSV: {output_csv}")


Processed designs: 30
Skipped entries: 1 (see Trat1_skipped_designs.log)
Wrote FASTA: Trat1_designs.fasta
Wrote CSV: Trat1_designs.csv
